# Notebook Sample: Marketing Investment Planning Preparation Analysis

| Doc Properties | Value |
| --- | --- |
| Prepared by | Julie  Koesmarno (signal@mssqlgirl.com) |
| Prepared on | Feb 17, 2020 |
| Purpose | To illustrate how to use Notebooks for Data Analysis. This notebook is created using [Azure Data Studio](https://docs.microsoft.com/en-us/sql/azure-data-studio/what-is?view=sql-server-ver15) |



## Abstract

Wide World Importers CEO is keen to understand the revenue, profit and margin spread of their products. This paper is to prepare for the next Marketing Planning Meeting with marketing investment as one of the topics. They would like to experiment and strategize on marketing investment. The idea is to work with two distinct product groups for marketing experimentation. 

## Description 
This document will use the Wide World Importers database to do analysis on product portfolio in the past year. 
We would like to identify two groups for planned experimentation in the next Marketing Planning Meeting: 
* **Group A**: After iterative discussions with marketing, we think that there are some stock items with high margin but low contribution to Total Revenue and Total Profit. 
* **Group B**: As part of the experimentation, we want to also understand how these stock items would perform against those with low margin and lower contribution to Total Revenue and Total Profit. 

Once we identify these groups, we will conclude with recommendation and potential next steps for experimentation especially what to track.

## Step 1 - Get and inspect the data

To access the Wide World Importers database:

| Server | Database | Credential |
| --- | --- | --- |
| 11.81.136.97,1433 | WideWorldImporters | Windows Auth

To get a copy of your own WideWorldImporters database, go to: https://github.com/Microsoft/sql-server-samples/releases/tag/wide-world-importers-v1.0 

Here's the assumption / description about the data that will be used in the analysis today.
* For the purpose of analysis prepared for Feb 2016 meeting, we will use the latest month data, i.e. Jan 2016. It is possible to extend this to show month over month trend.
* We will use Sales.Invoices and Sales.InvoiceLines which has information about Revenue for each invoice line item (i.e. ExtendedPrice) and Profit (i.e. LineProfit).
* Warehouse.Stock* tables have more information about the stock items. 
* No cumulative analysis needed for this document. 

> Note: A stock item can belong to many different stock groups. So it is important that throughout the analysis, when / if we want to do cumulative total or percentage of cumulative total, that we choose only one group that it belongs to "consistently".





### Show Top 10 rows from the tables that we will be using today. 

In [3]:
USE WideWorldImporters;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160101';
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

SELECT TOP 10
    --  COUNT(DISTINCT g.StockItemStockGroupID) StockGroups
     i.InvoiceDate
    ,i.InvoiceID
    ,si.StockItemName
    ,gr.StockGroupName
    ,il.ExtendedPrice AS Revenue
    ,il.LineProfit AS Profit
    ,il.Quantity
    ,il.LineProfit * 100.0 / il.ExtendedPrice AS [Margin%]    
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate
ORDER BY 
     i.InvoiceID
    ,i.InvoiceDate
    ,si.StockItemName

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.096

InvoiceDate,InvoiceID,StockItemName,StockGroupName,Revenue,Profit,Quantity,Margin%
2016-01-01,61321,"""The Gu"" red shirt XML tag t-shirt (White) XXS",Clothing,1490.40,792.00,72,53.1400966183574879
2016-01-01,61321,"""The Gu"" red shirt XML tag t-shirt (White) XXS",T-Shirts,1490.40,792.00,72,53.1400966183574879
2016-01-01,61321,"""The Gu"" red shirt XML tag t-shirt (White) XXS",Computing Novelties,1490.40,792.00,72,53.1400966183574879
2016-01-01,61322,Tape dispenser (Red),Packaging Materials,1840.00,750.00,50,40.7608695652173913
2016-01-01,61323,"""The Gu"" red shirt XML tag t-shirt (White) XS",Clothing,1242.00,660.00,60,53.1400966183574879
2016-01-01,61323,"""The Gu"" red shirt XML tag t-shirt (White) XS",T-Shirts,1242.00,660.00,60,53.1400966183574879
2016-01-01,61323,"""The Gu"" red shirt XML tag t-shirt (White) XS",Computing Novelties,1242.00,660.00,60,53.1400966183574879
2016-01-01,61324,Black and orange glass with care despatch tape 48mmx75m,Packaging Materials,510.60,252.00,120,49.3537015276145710
2016-01-01,61325,Shipping carton (Brown) 305x305x305mm,Packaging Materials,503.13,200.00,125,39.7511577524695406
2016-01-01,61326,"""The Gu"" red shirt XML tag t-shirt (White) 5XL",Clothing,745.20,360.00,36,48.3091787439613526


Show the number of unique stock items and unique stock item groups sold in January 2016.

In [45]:
USE WideWorldImporters;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160101'; 
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

SELECT 
     COUNT(DISTINCT si.StockItemName) UniqueStockItem
    ,COUNT(DISTINCT gr.StockGroupName) UniqueStockGroup
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate;

## Step 2 - Identify Group A

**Group A**: After iterative discussions with marketing, we think that there are some stock items with high margin but low contribution to Total Revenue and Total Profit. 

High margin refers to those that have higher than the Margin Percentage of all invoices in Jan 2016. 

In [46]:
USE WideWorldImporters;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160101'; 
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

DECLARE @TotalRevenue AS DECIMAL(18, 2);
DECLARE @TotalProfit AS DECIMAL(18, 2);
DECLARE @MarginPct AS DECIMAL(18,2);
DECLARE @LowProfitContribution AS DECIMAL(18,2) = 10;

SELECT
     @TotalRevenue  = SUM(il.ExtendedPrice)
    ,@TotalProfit   = SUM(il.LineProfit)
    ,@MarginPct     = SUM(il.LineProfit) * 100 / SUM(il.ExtendedPrice)
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate;

SELECT @TotalRevenue AS TotalRevenue, @TotalProfit AS TotalProfit, @MarginPct AS [Margin%];

SELECT 
    --  COUNT(DISTINCT g.StockItemStockGroupID) StockGroups
     gr.StockGroupName
    ,SUM(il.ExtendedPrice) Revenue
    ,SUM(il.LineProfit) Profit
    ,SUM(il.Quantity) Quantity
    ,SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) AS [Margin%]
    ,SUM(il.ExtendedPrice) * 100.0 / @TotalRevenue AS [%TotalRevenue]
    ,SUM(il.LineProfit) * 100.0 / @TotalProfit AS [%TotalProfit]
    ,COUNT(DISTINCT si.StockItemName) StockItems
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate
GROUP BY gr.StockGroupName
HAVING SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) > @MarginPct
AND SUM(il.LineProfit) * 100.0 / @TotalProfit < @LowProfitContribution
ORDER BY Revenue DESC;

## Step 3 - Identify Group B

**Group B**: As part of the experimentation, we want to also understand how these stock items would perform against those with low margin and lower contribution to Total Revenue and Total Profit. 

High margin refers to those that have higher than the Margin Percentage of all invoices in Jan 2016. 

In [47]:
USE WideWorldImporters;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160101'; 
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

DECLARE @TotalRevenue AS DECIMAL(18, 2);
DECLARE @TotalProfit AS DECIMAL(18, 2);
DECLARE @MarginPct AS DECIMAL(18,2);
DECLARE @LowRevenueContribution AS DECIMAL(18,2) = 5;

SELECT
     @TotalRevenue  = SUM(il.ExtendedPrice)
    ,@TotalProfit   = SUM(il.LineProfit)
    ,@MarginPct     = SUM(il.LineProfit) * 100 / SUM(il.ExtendedPrice)
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate;

SELECT @TotalRevenue AS TotalRevenue, @TotalProfit AS TotalProfit, @MarginPct AS [Margin%];

SELECT 
     gr.StockGroupName
    ,SUM(il.ExtendedPrice) Revenue
    ,SUM(il.LineProfit) Profit
    ,SUM(il.Quantity) Quantity
    ,SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) AS [Margin%]
    ,SUM(il.ExtendedPrice) * 100.0 / @TotalRevenue AS [%TotalRevenue]
    ,SUM(il.LineProfit) * 100.0 / @TotalProfit AS [%TotalProfit]
    ,COUNT(DISTINCT si.StockItemName) StockItems
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate
GROUP BY gr.StockGroupName
HAVING SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) < @MarginPct
AND SUM(il.ExtendedPrice) * 100.0 / @TotalRevenue > @LowRevenueContribution
ORDER BY Revenue DESC;

## Conclusion

Out of the 9 stock groups sold in January 2016, these are the proposed two groups that we can do marketing experimentation with:
1. **Group A** - Furry Footwear, USB Novelties and Mugs with percentage of total profit between 1.8% and 2.8%. 
2. **Group B** - Clothing, Novelty Items and Toys with percentagage of total profit between 6.5% and 24.4%

This paper includes their January baseline of the Quntity Sold, the Profit and the Revenue. The next analysis should include these metrics including the % month over month growth. 

